<a href="https://colab.research.google.com/github/santule/ERA/blob/main/S6/S6_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [153]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3,padding=1), # 16,28,28
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2), # 16,14,14
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3,padding=1), # 32,14,14
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2, 2), # 32,7,7
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 32, 3,padding=1), # 64,7,7
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 10, 3,padding=1), # 10,7,7
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm2d(10),
            nn.MaxPool2d(2, 2), # 10,3,3
        )

        self.conv4 = nn.Sequential(
            nn.AvgPool2d(3) # 10
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(-1,10)
        return F.log_softmax(x, dim=1)

In [154]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
           Dropout-3           [-1, 16, 28, 28]               0
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
            Conv2d-6           [-1, 32, 14, 14]           4,640
              ReLU-7           [-1, 32, 14, 14]               0
           Dropout-8           [-1, 32, 14, 14]               0
       BatchNorm2d-9           [-1, 32, 14, 14]              64
        MaxPool2d-10             [-1, 32, 7, 7]               0
           Conv2d-11             [-1, 32, 7, 7]           9,248
             ReLU-12             [-1, 32, 7, 7]               0
          Dropout-13             [-1, 32, 7, 7]               0
      BatchNorm2d-14             [-1, 3

In [155]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [158]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [159]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1, verbose=True)

for epoch in range(20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-02.


loss=0.17674826085567474 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.09it/s]



Test set: Average loss: 0.1067, Accuracy: 9728/10000 (97%)

Adjusting learning rate of group 0 to 1.0000e-02.


loss=0.023810168728232384 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.07it/s]



Test set: Average loss: 0.0559, Accuracy: 9854/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-02.


...


Test set: Average loss: 0.0200, Accuracy: 9939/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-03.


loss=0.007497437763959169 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.85it/s]



Test set: Average loss: 0.0189, Accuracy: 9946/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-03.


loss=0.039091240614652634 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.97it/s]



Test set: Average loss: 0.0201, Accuracy: 9942/10000 (99%)

Adjusting learning rate of group 0 to 1.0000e-03.
